# Data Cleaning
Die Rohdaten werden aus dem CSV eingelesen. Fehlende Daten werden entfernt und die Verspätung wird anhand 
von geplanter und tatsächlicher Abfahrtszeit berechnet.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load data with right types.
dtypes = {
    'Datum': str,
    'Wochentag': str,
    'Linienverlauf': str,
    'SollAb': str,
    'GPS-Breite Soll': np.float64,
    'GPS-Länge Soll': np.float64,
}

df_linie1_hin_1 = pd.read_csv(filepath_or_buffer='Echtzeitdaten_Linie1_hin_sheet1.csv', 
                              sep=';', dtype=dtypes, thousands=',')
df_linie1_hin_2 = pd.read_csv(filepath_or_buffer='Echtzeitdaten_Linie1_hin_sheet2.csv', 
                              sep=';', dtype=dtypes, thousands=',')
df_linie1_hin_3 = pd.read_csv(filepath_or_buffer='Echtzeitdaten_Linie1_hin_sheet3.csv', 
                              sep=';', dtype=dtypes, thousands=',')
df_linie1_hin_4 = pd.read_csv(filepath_or_buffer='Echtzeitdaten_Linie1_hin_sheet4.csv', 
                              sep=';', dtype=dtypes, thousands=',')
df_linie1_hin_1.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAn,IstAn,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll
0,10.10.2016,Montag,2,1081890,5556,30 Ludgeriplatz B,05:17:00,05:17:58,05:17:00,05:17:58,519558997.0,76278508.0
1,10.10.2016,Montag,2,1081890,5556,31 Goebenstr.,05:19:00,05:19:03,05:19:00,05:19:03,51952888.0,76254844.0
2,10.10.2016,Montag,2,1081890,5556,32 St. Joseph-Kirche,05:20:00,05:20:00,05:20:00,05:20:00,519493516.0,76243561.0
3,10.10.2016,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,05:21:00,05:20:44,05:21:00,05:20:44,519460313.0,76232794.0
4,10.10.2016,Montag,2,1081890,5556,34 Metzer Str.,05:23:00,05:21:44,05:23:00,05:23:20,519422225.0,76234541.0


In [3]:
# Check that parsing the right data types worked.
df_linie1_hin_1.dtypes

Datum               object
Wochentag           object
Fahrweg              int64
Fahrt                int64
Fahrzeugnummer       int64
Linienverlauf       object
SollAn              object
IstAn               object
SollAb              object
IstAb               object
GPS-Breite Soll    float64
GPS-Länge Soll     float64
dtype: object

In [4]:
# Combine the data frames into a single frame.
df_linie1_hin = pd.concat([df_linie1_hin_1, df_linie1_hin_2, df_linie1_hin_3, df_linie1_hin_4], 
                          ignore_index=True)
df_linie1_hin.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAn,IstAn,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll
0,10.10.2016,Montag,2,1081890,5556,30 Ludgeriplatz B,05:17:00,05:17:58,05:17:00,05:17:58,519558997.0,76278508.0
1,10.10.2016,Montag,2,1081890,5556,31 Goebenstr.,05:19:00,05:19:03,05:19:00,05:19:03,51952888.0,76254844.0
2,10.10.2016,Montag,2,1081890,5556,32 St. Joseph-Kirche,05:20:00,05:20:00,05:20:00,05:20:00,519493516.0,76243561.0
3,10.10.2016,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,05:21:00,05:20:44,05:21:00,05:20:44,519460313.0,76232794.0
4,10.10.2016,Montag,2,1081890,5556,34 Metzer Str.,05:23:00,05:21:44,05:23:00,05:23:20,519422225.0,76234541.0


In [5]:
# How many departure times are missing.
len(df_linie1_hin[df_linie1_hin['IstAb'] == ' '])

9820

In [6]:
# Drop rows with missing values.
df_linie1_hin = df_linie1_hin.drop(index=(df_linie1_hin[df_linie1_hin['IstAb'] == ' ']).index)
df_linie1_hin = df_linie1_hin.reset_index(drop=True)
len(df_linie1_hin[df_linie1_hin['IstAb'] == ' '])

0

In [7]:
# We still have enough values to work with.
df_linie1_hin.shape

(844111, 12)

In [8]:
# Convert departure times to datetime objects while setting the correct date.
df_linie1_hin.loc[:, 'SollAb'] = pd.to_datetime(df_linie1_hin['Datum'] + ' ' + df_linie1_hin['SollAb'], format='%d.%m.%Y %H:%M:%S')
df_linie1_hin.loc[:, 'IstAb'] = pd.to_datetime(df_linie1_hin['Datum'] + ' ' + df_linie1_hin['IstAb'], format='%d.%m.%Y %H:%M:%S')
df_linie1_hin.loc[:, 'Datum'] = pd.to_datetime(df_linie1_hin['Datum'], format='%d.%m.%Y')
df_linie1_hin.dtypes

Datum              datetime64[ns]
Wochentag                  object
Fahrweg                     int64
Fahrt                       int64
Fahrzeugnummer              int64
Linienverlauf              object
SollAn                     object
IstAn                      object
SollAb             datetime64[ns]
IstAb              datetime64[ns]
GPS-Breite Soll           float64
GPS-Länge Soll            float64
dtype: object

In [9]:
# Discard arrival times as they are not really important.
df_linie1_hin = df_linie1_hin.drop(columns=['SollAn', 'IstAn'])
df_linie1_hin.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll
0,2016-10-10,Montag,2,1081890,5556,30 Ludgeriplatz B,2016-10-10 05:17:00,2016-10-10 05:17:58,519558997.0,76278508.0
1,2016-10-10,Montag,2,1081890,5556,31 Goebenstr.,2016-10-10 05:19:00,2016-10-10 05:19:03,51952888.0,76254844.0
2,2016-10-10,Montag,2,1081890,5556,32 St. Joseph-Kirche,2016-10-10 05:20:00,2016-10-10 05:20:00,519493516.0,76243561.0
3,2016-10-10,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,2016-10-10 05:21:00,2016-10-10 05:20:44,519460313.0,76232794.0
4,2016-10-10,Montag,2,1081890,5556,34 Metzer Str.,2016-10-10 05:23:00,2016-10-10 05:23:20,519422225.0,76234541.0


In [10]:
# Calculate the delay.
df_linie1_hin['Verspätung'] = df_linie1_hin['IstAb'] - df_linie1_hin['SollAb']
df_linie1_hin.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll,Verspätung
0,2016-10-10,Montag,2,1081890,5556,30 Ludgeriplatz B,2016-10-10 05:17:00,2016-10-10 05:17:58,519558997.0,76278508.0,00:00:58
1,2016-10-10,Montag,2,1081890,5556,31 Goebenstr.,2016-10-10 05:19:00,2016-10-10 05:19:03,51952888.0,76254844.0,00:00:03
2,2016-10-10,Montag,2,1081890,5556,32 St. Joseph-Kirche,2016-10-10 05:20:00,2016-10-10 05:20:00,519493516.0,76243561.0,00:00:00
3,2016-10-10,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,2016-10-10 05:21:00,2016-10-10 05:20:44,519460313.0,76232794.0,-1 days +23:59:44
4,2016-10-10,Montag,2,1081890,5556,34 Metzer Str.,2016-10-10 05:23:00,2016-10-10 05:23:20,519422225.0,76234541.0,00:00:20


In [11]:
# Transform delay from timedelta to seconds.
df_linie1_hin['Verspätung_total_seconds'] = df_linie1_hin['Verspätung'].dt.total_seconds()
df_linie1_hin.head()

,Datum,Wochentag,Fahrweg,Fahrt,Fahrzeugnummer,Linienverlauf,SollAb,IstAb,GPS-Breite Soll,GPS-Länge Soll,Verspätung,Verspätung_total_seconds
0,2016-10-10,Montag,2,1081890,5556,30 Ludgeriplatz B,2016-10-10 05:17:00,2016-10-10 05:17:58,519558997.0,76278508.0,00:00:58,58.0
1,2016-10-10,Montag,2,1081890,5556,31 Goebenstr.,2016-10-10 05:19:00,2016-10-10 05:19:03,51952888.0,76254844.0,00:00:03,3.0
2,2016-10-10,Montag,2,1081890,5556,32 St. Joseph-Kirche,2016-10-10 05:20:00,2016-10-10 05:20:00,519493516.0,76243561.0,00:00:00,0.0
3,2016-10-10,Montag,2,1081890,5556,33 Alter Sch¿tzenhof A,2016-10-10 05:21:00,2016-10-10 05:20:44,519460313.0,76232794.0,-1 days +23:59:44,-16.0
4,2016-10-10,Montag,2,1081890,5556,34 Metzer Str.,2016-10-10 05:23:00,2016-10-10 05:23:20,519422225.0,76234541.0,00:00:20,20.0


In [12]:
# Save results.
df_linie1_hin.to_pickle('df_linie1_hin.pickle')